In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random

key = np.array([
            [1,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,0,0],
            [0,0,0,0,1,0,0,0],
            [0,0,1,0,0,0,1,0],
            [0,0,0,0,1,0,0,0],
            [0,0,0,0,0,0,1,0],
            [0,1,0,1,0,1,0,0]
        ])

def string_to_bin(message):
    bin_ = []
    for c in message:
        bin_.append('{0:08b}'.format(ord(c)))
    return bin_

def block_int_to_bin(block):
    arr_bin = ['{0:08b}'.format(int(e)) for e in block]
    np_bin = []
    for a in arr_bin:
        r = [int(b) for b in a]
        np_bin.append(r)
    return np.array(np_bin)

def block_bin_to_int(block):
    int_block = np.array([int(b, 2) for b in [''.join([str(c) for c in e]) for e in block]])
    return int_block

def random_position():
    return (random.randint(0,7),random.randint(0,7))

def draw(source):
    rgb = cv2.cvtColor(source, cv2.COLOR_BGR2RGB)
    plt.subplot(1,2,1)
    plt.imshow(rgb)

def hide_in_block(block, word):
    bin_word = '{0:08b}'.format(ord(word))
    print('Block:\n',block,'\nSecret word:',word,'====>',bin_word)
    for i in range(0,8):
        fi = block_int_to_bin(block[:,i])
        # SUM(fi and key)
        and_sum = np.bitwise_and(fi, key).sum()
        # SUM(key)
        key_sum = key.sum()
        print('\nSUM(fi and key):',and_sum,'| SUM(key):',key_sum,'| Bit:',bin_word[i])
        if and_sum > 0 and and_sum < key_sum:
            if and_sum % 2 == int(bin_word[i]):
                print('====> Do nothing')
            elif and_sum == 1:
                replace = False
                for y in range(0,8):
                    for x in range(0,8):
                        if fi[y,x] == 0 and key[y,x] == 1:
                            fi[y,x] = 1
                            block[:,i] = block_bin_to_int(fi)
                            replace = True
                            break
                        if replace is True:
                            break
                print('====> and_sum == 1')
            elif and_sum == key_sum - 1:
                replace = False
                for y in range(0,8):
                    for x in range(0,8):
                        if fi[y,x] == 1 and key[y,x] == 1:
                            fi[y,x] = 0
                            block[:,i] = block_bin_to_int(fi)
                            replace = True
                            break
                        if replace is True:
                            break
                print('====> and_sum == key_sum - 1')
            else:
                replace = False
                for y in range(0,8):
                    for x in range(0,8):
                        if key[y,x] == 1:
                            if fi[y,x] == 1:
                                fi[y,x] = 0
                            else:
                                fi[y,x] = 1
                            block[:,i] = block_bin_to_int(fi)
                            replace = True
                            break
                        if replace is True:
                            break
                print('====> Everything else')
    print('\nNew Block:\n', block)
    return block

# hide_in_block(
#     np.array(
#         [[143, 145, 147, 146, 147, 147, 148, 148],
#         [143, 144, 145, 145, 144, 145, 145, 146],
#         [144, 145, 146, 145, 144, 144, 146, 146],
#         [144, 144, 143, 145, 145, 146, 144, 146],
#         [144, 143, 145, 145, 145, 144, 145, 145],
#         [146, 145, 144, 144, 145, 148, 147, 146],
#         [147, 146, 145, 144, 145, 146, 148, 147],
#         [146, 146, 143, 145, 145, 146, 147, 147]]
#     ),
#     'd'
# )
# [[143,  17, 147, 146, 147, 147, 148,  20],
#  [143, 144, 145, 145, 144, 145, 145, 146],
#  [144, 145, 146, 145, 144, 144, 146, 146],
#  [144, 144, 143, 145, 145, 146, 144, 146],
#  [144, 143, 145, 145, 145, 144, 145, 145],
#  [146, 145, 144, 144, 145, 148, 147, 146],
#  [147, 146, 145, 144, 145, 146, 148, 147],
#  [146, 146, 143, 145, 145, 146, 147, 147]]


In [2]:
path = './cat.jpeg'
message = 'minhduc'
cover_image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
h = cover_image.shape[0]
w = cover_image.shape[1]
word_hidden = 0
done = False
# draw(cover_image)

In [3]:
for i in range(0, h, 8):
    for j in range(0, w, 8):
        block = cover_image[i:i+8,j:j+8]
        if block.shape[0] == 8 and block.shape[1] == 8:
            block = hide_in_block(block, message[word_hidden])
            cover_image[i:i+8,j:j+8] = block
            word_hidden += 1
        if word_hidden == len(message):
            done = True
            break
    if done is True:
        break
print("Hidden successfully")
# draw(cover_image)

Block:
 [[143 145 147 146 147 147 148 148]
 [143 144 145 145 144 145 145 146]
 [144 145 146 145 144 144 146 146]
 [144 144 143 145 145 146 144 146]
 [144 143 145 145 145 144 145 145]
 [146 145 144 144 145 148 147 146]
 [147 146 145 144 145 146 148 147]
 [146 146 143 145 145 146 147 147]] 
Secret word: m ====> 01101101

SUM(fi and key): 4 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 4 | SUM(key): 12 | Bit: 1
====> Everything else

SUM(fi and key): 3 | SUM(key): 12 | Bit: 1
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 1
====> Everything else

SUM(fi and key): 3 | SUM(key): 12 | Bit: 1
====> Do nothing

SUM(fi and key): 2 | SUM(key): 12 | Bit: 0
====> Do nothing

SUM(fi and key): 3 | SUM(key): 12 | Bit: 1
====> Do nothing

New Block:
 [[143  17 147 146  19 147 148 148]
 [143 144 145 145 144 145 145 146]
 [144 145 146 145 144 144 146 146]
 [144 144 143 145 145 146 144 146]
 [144 143 145 145 145 144 145

In [4]:
def retrieve_in_block(block):
    print('Retrieve in block:\n',block)
    bin_message = ''
    for i in range(0,8):
        fi = block_int_to_bin(block[:,i])
        # SUM(fi and key)
        and_sum = np.bitwise_and(fi, key).sum()
        # SUM(key)
        key_sum = key.sum()
        print('\nSUM(fi and key):',and_sum,'| SUM(key):',key_sum)
        if and_sum > 0 and and_sum < key_sum:
            if and_sum % 2 == 0:
                bin_message += "0"
            else:
                bin_message += "1"
    r_msg = chr(int(bin_message,2))
    print("Secret bin:",bin_message," | Character:", r_msg)
    return r_msg

# retrieve_in_block(
#     np.array([[143,  17, 147, 146, 147, 147, 148,  20],
#             [143, 144, 145, 145, 144, 145, 145, 146],
#             [144, 145, 146, 145, 144, 144, 146, 146],
#             [144, 144, 143, 145, 145, 146, 144, 146],
#             [144, 143, 145, 145, 145, 144, 145, 145],
#             [146, 145, 144, 144, 145, 148, 147, 146],
#             [147, 146, 145, 144, 145, 146, 148, 147],
#             [146, 146, 143, 145, 145, 146, 147, 147]])
# )


In [5]:
hidden_msg = ''
d = 10
for i in range(0, h, 8):
    for j in range(0, w, 8):
        block = cover_image[i:i+8,j:j+8]
        if block.shape[0] == 8 and block.shape[1] == 8:
            hidden_msg += retrieve_in_block(block)
            d -= 1
        if d == 0:
            break
    if d == 0:
        break
print('Hidden message is:',hidden_msg)


Retrieve in block:
 [[143  17 147 146  19 147 148 148]
 [143 144 145 145 144 145 145 146]
 [144 145 146 145 144 144 146 146]
 [144 144 143 145 145 146 144 146]
 [144 143 145 145 145 144 145 145]
 [146 145 144 144 145 148 147 146]
 [147 146 145 144 145 146 148 147]
 [146 146 143 145 145 146 147 147]]

SUM(fi and key): 4 | SUM(key): 12

SUM(fi and key): 3 | SUM(key): 12

SUM(fi and key): 3 | SUM(key): 12

SUM(fi and key): 2 | SUM(key): 12

SUM(fi and key): 1 | SUM(key): 12

SUM(fi and key): 3 | SUM(key): 12

SUM(fi and key): 2 | SUM(key): 12

SUM(fi and key): 3 | SUM(key): 12
Secret bin: 01101101  | Character: m
Retrieve in block:
 [[147  20  20  21  20  20 149 149]
 [146 146 147 148 147 149 149 148]
 [145 145 148 147 149 149 148 149]
 [148 146 148 147 147 149 149 149]
 [147 147 146 147 148 147 150 149]
 [147 148 146 148 148 149 147 151]
 [145 146 149 147 148 147 148 149]
 [148 147 148 148 147 150 150 149]]

SUM(fi and key): 4 | SUM(key): 12

SUM(fi and key): 3 | SUM(key): 12

SUM(fi and